# 5. Logging Data

In this tutorial, you will learn how to upload data to a MongoDB database directly from a microcontroller and read data from a database using Python.

<img src="../../_static/mongodb.png" width=80%>

*MongoDB database interface for a light-mixing database instance.*

## FAIR Data

Chemistry and materials research data is precious. By making your data **F**indable, **A**ccessible, **I**nteroperable, and **R**eusable ([FAIR](https://www.go-fair.org/fair-principles/)), you can maximize the impact of your research. Here are two conceptual examples of what FAIR data might look like in the physical sciences:

![FAIR data in materials science](./images/fair-data-materials-science.png)

*Defining FAIR Data in materials science. Reproduced from https://doi.org/10.1557/s43577-023-00498-4*

## Writing to MongoDB Using the Data API

We will be using [MongoDB](https://www.mongodb.com/), a popular ["NoSQL"](https://www.mongodb.com/nosql-explained) database, to store our data. MongoDB is a document-oriented database, which means that it stores data in JSON-like documents. MongoDB is a popular choice for IoT applications because it is easy to set up and use, and it is highly scalable. MongoDB also has a [Data API](https://docs.atlas.mongodb.com/data-api/) that allows you to read and write data directly from a microcontroller. For the purposes of this tutorial, a free-tier test database has been set up for you. Here, we provide you with an API key for test purposes. To avoid misuse that could arise from distributing a public API key (normally bad practice), only write permissions have been granted, and the database is set to autodelete entries once a certain storage threshold has been reached.

✅ Copy the following code into a new file called `write_mongodb.py` and run it on the microcontroller.

In [ ]:
# based on https://medium.com/@johnlpage/introduction-to-microcontrollers-and-the-pi-pico-w-f7a2d9ad1394
from netman import connectWiFi
import urequests

from my_secrets import SSID, PASSWORD, COURSE_ID

connectWiFi(SSID, PASSWORD, country="US")

API_KEY = "UT4cdinBetBaNqCBc5hISkaArhllv5dWfzXgbYsLYzpv79nqNhVwVsudQU5ZUmBE"  # Public API key for demo purposes only
CLUSTER_NAME = "test-cluster"
DATABASE_NAME = "test-db"
COLLECTION_NAME = "write-to-me"

URL_ENDPOINT = (
    "https://us-east-2.aws.data.mongodb-api.com/app/data-ibmqs/endpoint/data/v1/action/insertOne"
)

headers = {"api-key": API_KEY}
document = {"course_id": COURSE_ID}

payload = {
    "dataSource": CLUSTER_NAME,
    "database": DATABASE_NAME,
    "collection": COLLECTION_NAME,
    "document": document,
}

print(f"sending document to {CLUSTER_NAME}:{DATABASE_NAME}:{COLLECTION_NAME}")

num_retries = 3
for _ in range(num_retries):
    response = urequests.post(URL_ENDPOINT, headers=headers, json=payload)
    txt = str(response.text)
    status_code = response.status_code

    print(f"Response: ({status_code}), msg = {txt}")

    response.close()

    if status_code == 201:
        print("Added Successfully")
        break

    print("Retrying...")

The output should look something like the following (MAC address and IP address redacted, and your insertedId will be different):

```python
MAC address: ***
connected
ip = ***
sending document to test-cluster:test-db:write-to-me
Response: (201), msg = {"insertedId":"6594bfbfb3c925d2fdfbb7e8"}
Added Successfully
```

## Reading from MongoDB Using PyMongo

✅ Run the code from [the companion notebook](./1.5.1-pymongo.ipynb) to read data from a MongoDB database.

<!-- ✅ Copy the following code into a new file called `read_mongodb.py` and run it on the microcontroller. -->

In [ ]:
import urequests
import ujson

# Set up the database connection
url = "https://api.mongodb.com"
project_id = "YOUR_PROJECT_ID"
database = "YOUR_DATABASE_NAME"
collection = "YOUR_COLLECTION_NAME"
username = "YOUR_USERNAME"
password = "YOUR_PASSWORD"

# Set up the query
query = {"_id": "YOUR_DOCUMENT_ID"}

# Set up the request
url = "{}/v2/project/{}/database/{}/collection/{}".format(url, project_id, database, collection)
headers = {"Content-Type": "application/json"}
auth = (username, password)

# Make the request
response = urequests.get(url, headers=headers, auth=auth, json=query)

# Parse the response
data = ujson.loads(response.text)
print(data)

In [ ]:
# https://github.com/sparks-baird/self-driving-lab-demo/blob/24146ae3078972e8ae2ca74f0428794906a13797/src/public_mqtt_sdl_demo/scripts/data_logging_basic.py

from secrets import (
    DEVICE_NICKNAME,
    MONGODB_API_KEY,
    MONGODB_COLLECTION_NAME,
    PASSWORD,
    SSID,
)

from data_logging import log_to_mongodb
from netman import connectWiFi

connectWiFi(SSID, PASSWORD, country="US")

mongodb_app_name = "data-sarkl"
mongodb_url = f"https://data.mongodb-api.com/app/{mongodb_app_name}/endpoint/data/v1/action/insertOne"  # noqa: E501
mongodb_cluster_name = "sparks-materials-informatics"
mongodb_database_name = "clslab-light-mixing"

log_to_mongodb(
    {"device_nickname": DEVICE_NICKNAME},
    api_key=MONGODB_API_KEY,
    collection_name=MONGODB_COLLECTION_NAME,
    url=mongodb_url,
    cluster_name=mongodb_cluster_name,
    database_name=mongodb_database_name,
    verbose=True,
    retries=2,
)


## Reading Material

✅ Read [Community action on FAIR data will fuel a revolution in materials research](https://doi.org/10.1557/s43577-023-00498-4)

✅ Watch [the following video](https://youtu.be/cdSENQPsAiI?si=KkwmGzoQmQP-CsQo) about materials databases:

<iframe width="560" height="315" src="https://www.youtube.com/embed/cdSENQPsAiI?si=h1dw65aA1tNamyQV" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe>

## Additional Resources
- [MongoDB Getting Started Interactive Tutorial](https://www.mongodb.com/docs/manual/tutorial/getting-started/)
- [PyMongo Project-based Tutorial using FastAPI](https://www.mongodb.com/languages/python/pymongo-tutorial)
- [PyMongo Docs Tutorial](https://pymongo.readthedocs.io/en/stable/tutorial.html)
- [Connecting a Raspberry Pi Pico W to MongoDB Atlas](https://medium.com/@johnlpage/introduction-to-microcontrollers-and-the-pi-pico-w-f7a2d9ad1394)
- [FAIR data principles](https://www.go-fair.org/fair-principles/) (see also https://doi.org/10.1038/sdata.2016.18)
- [Video: Sparks Materials Informatics, Materials Databases](https://www.youtube.com/watch?v=cdSENQPsAiI)

<!-- - [IUPAC FAIR Chemistry Cookbook](https://iupac.github.io/WFChemCookbook/intro.html) TOO "HALF-BAKED" -->